# Квадратурные формулы. Наивысшая АСТ.

## Формула Гаусса

$p(x)\equiv1, \, [a, b] = [-1, 1]$

$\int_{-1}^1f(x)dx$

### Полином Лежандра

$P_n(x) = \frac {\displaystyle 1} {\displaystyle 2^n n!} \frac {\displaystyle d^n} {\displaystyle dx^n} (x^2 - 1)^n$

Корни полинома -- узлы в формуле Гауса.

$dx \qquad x = \frac {a-b} {2} t + \frac {b+a} {2} \\
dx = \frac {a-b} {2} dt$

$\int_a^b f(x) = \frac{b-a} {2} \int_{-1}^1 f(\frac {b-a} {2} t + \frac {b+a} {2})dt = \frac{b-a} {2} \sum_{k=1}^n A_k f(\frac {b-a} {2} t_k + \frac {b+a} {2})$

$A_k$ и $t_k$ -- табличные значения. 

## Задача №1 Формула Гаусса

$f(x)=\frac {1} {x^2 + 4} \qquad [0,1]$

1. Применить формулу Гаусса для $n=5$
2. Посчитать фактическую погрешность
3. Сравнить с квадратурной формулой Симпсона $N=2, \, h=\frac {b-a} {2N}$ по 5 узлов.

In [1]:
let ts = [-0.906179845938664; -0.538469310105683; 0.; 0.538469310105683; 0.906179845938664]
let As = [0.236926885056189; 0.478628670499366; 0.568888888888889; 0.478628670499366; 0.236926885056189]

In [2]:
let a = 0.
let b = 1.
let N = 2
let h = (b - a) / 2. / float N
let n = 5

In [3]:
let f x = 1. / (x ** 2. + 4.)

In [4]:
let gauss = (b - a) / 2. * (List.fold2 (fun acc t A -> acc + A * f ((b - a) / 2. * t + (b + a) / 2.)) 0. ts As)

### Точное значение

$\int_0^1 \frac {1}{x^2 + 4} dx = \frac 1 2 \arctan {\frac x 2} |_0^1 = \frac 1 2 \arctan \frac 1 2$

In [5]:
let exact = atan (1. / 2.) / 2.

In [6]:
abs (exact - gauss)

1.890491375e-10

### Симпсона

$h=\frac{b-a}{2N} \qquad [x_{k-1}, x_{k+1}] \, k = 0, \dots , N$

$\int_a^b f(x) dx = \frac {b-a}{6N} (f_0 + 4(f_1 + f_3 + \cdots + f_{2N-1}) + 2(f_2 + f_4 + \cdots + f_{2N-2}) + f_{2N})$

In [7]:
let xk k = a + k * h

In [8]:
let simpson = 
    let term n =
        match n % 2 = 0 with
        | true  -> 2. * f (xk (float n))
        | false -> 4. * f (xk (float n))
    h / 3. * (f (xk 0.) + f (xk (2. * (float N))) 
        + ([1 .. 2 * N - 1] |> List.fold (fun acc k -> acc + term k) 0.))

In [9]:
abs (exact - simpson)

2.524555982e-06

## Задача №2 

Дано: весовая функция $p(x) = \sqrt{1 - x}$, 3 узла $0, 0.5, 1$, интеграл функции $f(x) = e^{x} \cdot \sqrt {1 - x}$ на промежутке $[0,1]$

1. Находим точное значение.
2. Строим интерполяционную квадратурную форму с весом $p(x)$ по данным узлам.
3. Считаем погрешность.
4. Строим квадратурную форму наивысшего АСТ $n=3$.
5. Фактическая погрешность, сравниваем с фиксированными узлами.

In [10]:
let f x = exp(x)*sqrt(1. - x)
let realAnswer = 1.030078469278704975539089925665950448932564589318472524730
let xs = [0.; 0.5; 1.]

$A_k = \int_a^b p(x) l_k = \int_a^b p(x) \frac {w(x)} {(x-x_k)w'(x)}$

$w(x)= \prod_{k=1}^n (x-x_k)$

$l_k(x)=\frac{\displaystyle w(x)}{\displaystyle(x-x_k)w(x_k)}=\prod_{i \ne k}\frac{\displaystyle x-x_i}{\displaystyle x_k-x_i}$

$l_0(x) = 2(x - 0.5)(x - 1)$

$l_1(x) = -4x(x - 1)$

$l_2(x) = 2x(x - 0.5)$

$A_0 = \int_0^1 \sqrt {1 - x} \cdot 2(x - 0.5)(x - 1)dx = 0.171429$

$A_1 = \int_0^1 \sqrt {1 - x} \cdot -4x(x - 1)dx = \frac 4 3$

$A_2 = \int_0^1 \sqrt {1 - x} \cdot 2x(x - 0.5)dx = 0.0380952$

In [11]:
let As = [0.171429; 4. / 3.; 0.0380952]

$\int_a^b p(x) f(x) dx = \sum_{k=0}^{n-1} A_k f(x_k)$

In [12]:
let interpol = List.fold2 (fun acc A x -> acc + A * f x) 0. As xs

In [13]:
abs (realAnswer - interpol)

0.6957798518

### Поиск наивысшей АСТ

$\mu_k = \int_a^b p(x)x^kdx$

$\mu_0 = \int_0^1 \sqrt {1 - x} dx = \frac {2} {3}$

$\mu_1 = \int_0^1 \sqrt {1 - x} \cdot x dx = \frac {4} {15}$

$\mu_2 = \int_0^1 \sqrt {1 - x} \cdot x^2 dx = \frac {16} {105}$

$\mu_3 = \int_0^1 \sqrt {1 - x} \cdot x^3 dx = \frac {32} {315}$

$\mu_4 = \int_0^1 \sqrt {1 - x} \cdot x^4 dx = \frac {256} {3465}$

$\mu_5 = \int_0^1 \sqrt {1 - x} \cdot x^5 dx = \frac {512} {9009}$

$w(x) = x^3 + a_1 x^2 + a_2 x + a_3$

$a_1\mu_2 + a_2 \mu_1 + a_3 \mu_0 = -\mu_3$

$a_1\mu_3 + a_2 \mu_2 + a_3 \mu_1 = -\mu_4$

$a_1\mu_4 + a_2 \mu_3 + a_3 \mu_2 = -\mu_5$

$a_1 = -\frac {18} {13}$

$a_2 = \frac {72} {143}$

$a_3 = -\frac {16} {429}$

Корни $w(x)$

In [14]:
let xs = [0.099194; 0.45013; 0.83529]

$l_0(x) = \frac{(x - 0.45013)(x - 0.83529)}{0.258322585856}$

$l_1(x) = \frac{(x - 0.099194)(x - 0.83529)}{-0.13516650976}$

$l_2(x) = \frac{(x - 0.45013)(x - 0.099194)}{0.28351473536}$

In [15]:
let As = [0.233281; 0.307603; 0.125783]

In [16]:
let greatAST = List.fold2 (fun acc A x -> acc + A * f x) 0. As xs

In [17]:
abs (realAnswer - greatAST)

0.310115195